## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Balancer

https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum

06/09/2022 Price impact still needed

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum'

In [3]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

True

6-Month Average Daily Volume

In [4]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [5]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [6]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [8]:
df['dailyVolumeUSD'].astype(float).mean()

89546052.40941167

6-Month Average TVL

In [9]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/balancer.csv")

In [10]:
lma

,Unnamed: 0,Date,Timestamp,Balancer,Balancer.1,Balancer.2,Balancer.3,Balancer.4,Balancer.5,Balancer.6,Balancer.7,Balancer.8,Balancer.9,Balancer.10,Balancer.11
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,ethereum,polygon,arbitrum,Total,ethereum,polygon,arbitrum,Total,ethereum,polygon,arbitrum,Total
2,NaN,NaN,NaN,TVL,TVL,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USDT,USDT,USDT,USDT,USDT,USDT,USDT,USDT
4,NaN,24/03/2020,1.585008e+09,7833.948843186513,NaN,NaN,7833.948843186513,7833.948843186513,NaN,NaN,7833.948843186513,7808.045244,NaN,NaN,7808.045244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,NaN,02/09/2022,1.662077e+09,1549684069.427236,62754098.087032,13368539.162001,1625806706.676269,1549684069.427236,62754098.087032,13368539.162001,1625806706.676269,1549684069.427236,62754098.087032,13368539.162001,1625806706.676269
896,NaN,03/09/2022,1.662163e+09,1535451483.2507179,63447447.130664594,13164413.901126487,1612063344.2825089,1535451483.2507179,63447447.130664594,13164413.901126487,1612063344.2825089,1533917565.685033,63384063.067597,13151262.638488,1610452891.391118
897,NaN,04/09/2022,1.662250e+09,1477124788.9708078,63872848.249165945,12990612.84019599,1553988250.0601697,1477124788.9708078,63872848.249165945,12990612.84019599,1553988250.0601697,1475649139.830977,63809039.209956,12977635.204991,1552435814.245924
898,NaN,05/09/2022,1.662336e+09,1531002701.0175686,66757736.028127134,13016076.236632468,1610776513.2823284,1531002701.0175686,66757736.028127134,13016076.236632468,1610776513.2823284,1529473227.789779,66691044.983144,13003073.163469,1609167345.936392


In [11]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

2011415420.2205076

Largest Pool Concentration

In [12]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_106821/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [13]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0xf3799cbae9c028c1a590f4463dff039b8f4986be,None,0,0,1618889021,12274556
1,0x61d5dc44849c9c87b0856a2a311536205c96c7fd,Comfy,21785.442774692943810205251199,12755999.1661168152963283538616655,1619037254,12285749
2,0x647c1fd457b95b75d0972ff08fe01d7d7bda05df,Balancer 80 BAL 20 WETH,22.780064571369506561003928,1681.389832680707987186142857,1619044051,12286257
3,0x0297e37f1873d2dab4487aa67cd56b58e2f27875,Balancer 50 WBTC 50 WETH,458.51106446778255370338,9553.038849894420661988930759,1619044189,12286276
4,0xc6a5032dc4bf638e15b4a66bc718ba7ba474ff73,Balancer 60 WETH 40 DAI,1583.986468282437688425071303,8304.202057513854709636521029,1619188473,12297008
...,...,...,...,...,...,...
854,0xcb146c5fccccf4d444e36e158f4167181a752721,ZEON-USDT,158.38033839679614607818834,50.0590470960491433338604095,1662031599,15452599
855,0xbb692c7b6087f390930b13e57c080ea215c184fe,SILKRD Copper LBP,6437.770211752099,1820.8909843532405,1662075500,15455776
856,0x563bb570107fc2d2de54caac58c91e4f27d68153,EU Copper LBP,0.000000998201,0,1662116923,15458798
857,0xc3df7970e204d26d6b6fb3bd720f177eb2f8117e,GETH Copper LBP,3404.610292933385,199.1431959179425,1662235713,15467386


In [14]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.2402164296104777

In [15]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.8055463874049088

Liquidity Pools with > $1mn TVL

In [16]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

44

Monthly Active Users

In [17]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [18]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [19]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [21]:
df['dailyActiveUsers'].astype(float).mean()

823.2185792349727

Implied slippage